In [1]:
import os

In [4]:
%pwd

'g:\\End-to-End-ML-Project\\End-to-end-Ml-project'

In [3]:
os.chdir('../')

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass (frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories


In [ ]:
class ConfigurationManager:
    #constants dir contains path of config, paarams, schema
    def __init__(self, config_pathfile = CONFIG_PATH_FILE, schema_pathfile = SCHEMA_PATH_FILE, 
                 params_pathfile = PARMAS_PATH_FILE):      
            
        # read_yaml function read the yaml file & return it into dictionary using ConfigBox
        self.config = read_yaml(config_pathfile) 
        self.params = read_yaml(params_pathfile)
        self.schema = read_yaml(schema_pathfile)

        # create_directories will create directory inside config.yaml under key artifacts_root
        create_directories([self.config.artifacts_root])
    
    # function get_data_ingestion_config will values under key data_ingestion
    # & create directiory & also return the values in function DataIngestionConfig
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir
            train_data_path = config.train_data_path
            test_data_path= config.test_data_path
            model_name = config.model_name
            alpha = params.alpha
            l1_ratio = params.l1_ratio
            target_column = schema.name      
            )
        return model_trainer_config

In [7]:
from sklearn.linear_model import ElasticNet
import joblib
import os
from mlproject import logger
import pandas as pd

In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha = self.config.alpha, l1_ratio = self.config.l1_ratio, random_state=42)
        lr.fit(X_train, y_train)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e